In [ ]:
import os

from dotenv import load_dotenv
from llama_index.core import Settings, StorageContext, VectorStoreIndex
from llama_index.core.chat_engine import ContextChatEngine
from llama_index.readers.file import DocxReader
from llama_index.readers.google import GoogleDriveReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import CollectionInfo, Distance, VectorParams

from codified_client import CodifiedClient, CodifiedRetriever, access_context

load_dotenv()

dimensions = len(Settings.embed_model.get_text_embedding("get vector dimensions"))

In [ ]:
# Find or create a Qdrant collection

In [2]:
def create_qdrant_collection(qdrant_client: QdrantClient) -> CollectionInfo:
    qdrant_client = QdrantClient(
        url=os.environ["QDRANT_URL"],
        api_key=os.environ["QDRANT_API_KEY"]
    )

    collection_name = os.environ["QDRANT_COLLECTION_NAME"]

    if not qdrant_client.collection_exists(collection_name):
        qdrant_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=dimensions,
                distance=Distance.COSINE
            )
        )

    return qdrant_client.get_collection(
        collection_name=collection_name
    )

qdrant_client = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_KEY"]
)

collection = create_qdrant_collection(qdrant_client)

vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=os.environ["QDRANT_COLLECTION_NAME"],
)


# Load documents then create and store embeddings.

In [33]:
storage_ctx=StorageContext.from_defaults(vector_store=vector_store)

reader = GoogleDriveReader(
    folder_id=os.environ["DRIVE_OR_FOLDER_ID"],
    service_account_key_path="gdrive-creds.json",
    file_extractor={"docx": DocxReader()}
)

index = VectorStoreIndex.from_documents(reader.load_data(), storage_context=storage_ctx)
print(index)

# Simple llama-based chat

In [3]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
ce = index.as_chat_engine()
response = ce.chat("What property is Emily Carter leasing or sub-leasing?")
print(response)

Emily Carter is subleasing the property located at 123 Birch Street, Newark, Delaware to Michael Brown.


# Create a codified retriever wrapping the default index retriever

In [3]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
codified_client = CodifiedClient(os.environ["CODIFIED_API_KEY"], os.environ["CODIFIED_URL"])
codified_retriever = CodifiedRetriever(codified_client, index.as_retriever())
ce = ContextChatEngine.from_defaults(
    retriever=codified_retriever
)

# test prompts with different access contexts

In [6]:
with access_context(user_email="stefan@codified.io"):
    response = ce.chat("What property is Emily Carter leasing or sub-leasing?")
    print(response)

with access_context(user_email="camila.c@g3a.io"):
    response = ce.chat("What property is Emily Carter leasing or sub-leasing?")
    print(response)

Emily Carter is subleasing the property located at 123 Birch Street, Newark, Delaware.
Empty Response
